In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 1, 15),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 1, 15), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1494497, 124), (1494497, 4), (683009, 124), (683009, 4))

In [4]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((124, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, 'relu'))
model.add(Dense(1, 'linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 flatten (Flatten)           (None, 7808)              0         
                                                                 
 dense (Dense)               (None, 8)                 62472     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 62,737
Trainable params: 62,737
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/d-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])
model_view.save('../model/saved/d-view.keras')


Epoch 1/10
46704/46704 [==============================] - 173s 4ms/step - loss: 16.7790 - root_mean_squared_error: 4.0962 - val_loss: 80.8011 - val_root_mean_squared_error: 8.9889
Epoch 2/10
46704/46704 [==============================] - 165s 4ms/step - loss: 14.8952 - root_mean_squared_error: 3.8594 - val_loss: 59.0256 - val_root_mean_squared_error: 7.6828
Epoch 3/10
46704/46704 [==============================] - 162s 3ms/step - loss: 14.1313 - root_mean_squared_error: 3.7592 - val_loss: 84.3055 - val_root_mean_squared_error: 9.1818
Epoch 4/10
46704/46704 [==============================] - 165s 4ms/step - loss: 13.7947 - root_mean_squared_error: 3.7141 - val_loss: 81.2648 - val_root_mean_squared_error: 9.0147
Epoch 5/10
46704/46704 [==============================] - 166s 4ms/step - loss: 13.6809 - root_mean_squared_error: 3.6988 - val_loss: 90.7437 - val_root_mean_squared_error: 9.5259
Epoch 6/10
46704/46704 [==============================] - 166s 4ms/step - loss: 13.6153 - root_mean_

In [6]:

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/d-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])
model_cart.save('../model/saved/d-cart.keras')


Epoch 1/10
46704/46704 [==============================] - 181s 4ms/step - loss: 9.6037 - root_mean_squared_error: 3.0990 - val_loss: 38.3396 - val_root_mean_squared_error: 6.1919
Epoch 2/10
46704/46704 [==============================] - 148s 3ms/step - loss: 7.9395 - root_mean_squared_error: 2.8177 - val_loss: 36.6714 - val_root_mean_squared_error: 6.0557
Epoch 3/10
46704/46704 [==============================] - 147s 3ms/step - loss: 7.6835 - root_mean_squared_error: 2.7719 - val_loss: 36.4282 - val_root_mean_squared_error: 6.0356
Epoch 4/10
46704/46704 [==============================] - 147s 3ms/step - loss: 7.5108 - root_mean_squared_error: 2.7406 - val_loss: 38.2460 - val_root_mean_squared_error: 6.1843
Epoch 5/10
46704/46704 [==============================] - 146s 3ms/step - loss: 7.4081 - root_mean_squared_error: 2.7218 - val_loss: 40.6916 - val_root_mean_squared_error: 6.3790
Epoch 6/10
46704/46704 [==============================] - 150s 3ms/step - loss: 7.2918 - root_mean_square

In [7]:

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/d-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])
model_remove_from_cart.save('../model/saved/d-remove_from_cart.keras')


Epoch 1/10
46704/46704 [==============================] - 166s 4ms/step - loss: 4.1772 - root_mean_squared_error: 2.0438 - val_loss: 37.4937 - val_root_mean_squared_error: 6.1232
Epoch 2/10
46704/46704 [==============================] - 141s 3ms/step - loss: 3.4331 - root_mean_squared_error: 1.8529 - val_loss: 36.7340 - val_root_mean_squared_error: 6.0609
Epoch 3/10
46704/46704 [==============================] - 143s 3ms/step - loss: 3.3399 - root_mean_squared_error: 1.8275 - val_loss: 36.1196 - val_root_mean_squared_error: 6.0100
Epoch 4/10
46704/46704 [==============================] - 152s 3ms/step - loss: 3.3036 - root_mean_squared_error: 1.8176 - val_loss: 36.8385 - val_root_mean_squared_error: 6.0695
Epoch 5/10
46704/46704 [==============================] - 142s 3ms/step - loss: 3.2752 - root_mean_squared_error: 1.8098 - val_loss: 37.2262 - val_root_mean_squared_error: 6.1013
Epoch 6/10
46704/46704 [==============================] - 149s 3ms/step - loss: 3.2590 - root_mean_square

In [8]:

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/d-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])
model_purchase.save('../model/saved/d-purchase.keras')


Epoch 1/10
46704/46704 [==============================] - 166s 4ms/step - loss: 1.2943 - root_mean_squared_error: 1.1377 - val_loss: 41.3845 - val_root_mean_squared_error: 6.4331
Epoch 2/10
46704/46704 [==============================] - 156s 3ms/step - loss: 0.9055 - root_mean_squared_error: 0.9516 - val_loss: 43.9836 - val_root_mean_squared_error: 6.6320
Epoch 3/10
46704/46704 [==============================] - 156s 3ms/step - loss: 0.8360 - root_mean_squared_error: 0.9143 - val_loss: 41.9207 - val_root_mean_squared_error: 6.4746
Epoch 4/10
46704/46704 [==============================] - 157s 3ms/step - loss: 0.7898 - root_mean_squared_error: 0.8887 - val_loss: 41.7262 - val_root_mean_squared_error: 6.4596
Epoch 5/10
46704/46704 [==============================] - 158s 3ms/step - loss: 0.7607 - root_mean_squared_error: 0.8722 - val_loss: 43.4735 - val_root_mean_squared_error: 6.5934
Epoch 6/10
46704/46704 [==============================] - 156s 3ms/step - loss: 0.7408 - root_mean_square